<a href="https://colab.research.google.com/github/georgemoum-cylearn/gmlearn/blob/main/Threat_Hunting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#εγκατάσταση πακέτου pycaret (https://pycaret.org/)
!pip install pycaret
#εγκατάσταση πακέτου dataprep (https://dataprep.ai/)
!pip install dataprep
#***ΠΡΟΣΟΧΗ*** μετά την εγκατάσταση χρειάζεται restart το session
#***θα σας ζητηθεί επανεκκίνηση με σχετικό κόκκινο μήνυμα μετά την ολοκλήρωση της εγκατάστασης
#***Πατήστε το κουμπί RESTART SESSION στο σχετικό μήνυμα

  Using cached dataprep-0.4.1-py3-none-any.whl.metadata (14 kB)
  Using cached bokeh-2.4.3-py3-none-any.whl.metadata (14 kB)
  Using cached dask-2.30.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached Flask-1.1.4-py2.py3-none-any.whl.metadata (4.6 kB)
  Using cached Flask_Cors-3.0.10-py2.py3-none-any.whl.metadata (5.4 kB)
  Using cached Jinja2-2.11.3-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached jsonpath_ng-1.7.0-py3-none-any.whl.metadata (18 kB)
  Using cached levenshtein-0.12.0.tar.gz (54 kB)
  Preparing metadata (setup.py) ... done
  Using cached Metaphone-0.6.tar.gz (14 kB)
  Preparing metadata (setup.py) ... done
  Using cached pandas-1.5.3.tar.gz (5.2 MB)
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel 

In [4]:
#εισαγωγή βιβλιοθηκών - πακέτων
import pandas as pd
import matplotlib.pyplot as plt

 Εγκατάσταση του pyod για να διασφαλιστεί η διαθεσιμότητα του IForest
!pip install pyod

 from pycaret.regression import from dataprep.eda import create_report
from pyod.models.iforest import IForest
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
!pip install dataprep




  Using cached dataprep-0.4.1-py3-none-any.whl.metadata (14 kB)
  Using cached bokeh-2.4.3-py3-none-any.whl.metadata (14 kB)
  Using cached dask-2.30.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached Flask-1.1.4-py2.py3-none-any.whl.metadata (4.6 kB)
  Using cached Flask_Cors-3.0.10-py2.py3-none-any.whl.metadata (5.4 kB)
  Using cached Jinja2-2.11.3-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached jsonpath_ng-1.7.0-py3-none-any.whl.metadata (18 kB)
  Using cached levenshtein-0.12.0.tar.gz (54 kB)
  Preparing metadata (setup.py) ... done
  Using cached Metaphone-0.6.tar.gz (14 kB)
  Preparing metadata (setup.py) ... done
  Using cached pandas-1.5.3.tar.gz (5.2 MB)
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel 

In [7]:
#εισαγωγή του συνόλου δεδομένων
url = "https://raw.githubusercontent.com/kdemertzis/EKPA/main/Data/Network-flows.csv" # Παράδειγμα URL, αν έχετε δικό σας, αντικαταστήστε το
df = pd.read_csv(url)
df.shape

(155123, 19)

In [8]:
#ανακάτεμα των δεδομένων
df = df.sample(frac = 1)

In [9]:
#οπτικοποίηση συνόλου δεδομένων
df.head()

,src_ip,src_port,dst_ip,dst_port,proto,pktTotalCount,octetTotalCount,min_ps,max_ps,avg_ps,std_dev_ps,flowStart,flowEnd,min_piat,max_piat,avg_piat,std_dev_piat,flowEndReason,flowDuration
54479,192.168.122.103,53500,31.13.67.11,443,6,13,3374,52,2520,259.538462,654.811261,1.559766e+09,1.559766e+09,0.000033,0.116167,0.010696,0.031820,3,0.128347
17088,192.168.121.3,50043,185.34.194.66,443,6,38,22708,40,2836,597.578947,748.924760,1.559770e+09,1.559770e+09,0.000002,6.406853,0.296933,1.115577,3,10.986507
89801,192.168.127.33,65262,172.217.2.142,443,6,31,9481,40,1458,305.838710,467.485111,1.559765e+09,1.559765e+09,0.000003,0.966482,0.041602,0.173826,3,1.248046
3546,192.168.128.207,50045,52.114.32.8,443,6,20,8314,40,3827,415.700000,914.428516,1.559659e+09,1.559659e+09,0.000002,0.509672,0.057948,0.131498,3,1.101018
56418,192.168.122.106,58748,172.16.255.200,53,17,2,215,66,149,107.500000,41.500000,1.559771e+09,1.559771e+09,0.000416,0.000416,0.000416,0.000000,2,0.000416


In [10]:
#Διερευνητική ανάλυση δεδομένων (Exploratory Data Analysis - EDA)
# create_report(df)

In [11]:

#διαχωρισμός των δεδομένων σε σύνολα εκπαίδευσης (train) και δοκιμών (test)
train_size = int(0.8 * len(df))
train_df = df[:train_size]
test_df = df[train_size:]

In [12]:
#Εκκίνηση της ρύθμισης του PyCaret με προεπεξεργασία δεδομένων
# reg_setup = setup(data=df, target='flowDuration', session_id=42,
#                   normalize=True, transformation=True,
#                   ignore_features=['proto'])

In [13]:
#σύγκριση των διαθέσιμων μοντέλων ΜΜ
# best = compare_models(cross_validation=False, n_select = 1)
# %load_ext tensorboard

In [14]:
#αναφορά των υπερπαραμέτρων του μοντέλου με την υψηλότερη απόδοση
# print(best)

In [15]:
#μέτρα αξιολόγησης του μοντέλου με την υψηλότερη απόδοση
# evaluate_model(best)

In [23]:
#δημιουργία μοντέλου ανίχνευσης ανωμαλιών Isolation Forest
iforest = IForest(contamination=0.1)
iforest.fit(train_df[['flowDuration']])
model = RandomForestClassifier(n_estimators=100, random_state=42)
clf = DecisionTreeClassifier(random_state=42)

In [21]:
#πίνακας προβλέψεων με ανωμαλίες στο σύνολο δοκιμών
test_df['anomaly'] = iforest.predict(test_df[['flowDuration']])

/usr/local/lib/python3.12/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
/tmp/ipython-input-3025008995.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['anomaly'] = iforest.predict(test_df[['flowDuration']])


In [18]:
#δημιουργία νέας στήλης 'Label' με βάση το αποτέλεσμα του ελέγχου ανίχνευσης ανωμαλιών
test_df['Label'] = test_df['anomaly'].apply(lambda x: 'Anomaly' if x == 1 else 'Normal')

/tmp/ipython-input-2873243607.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Label'] = test_df['anomaly'].apply(lambda x: 'Anomaly' if x == 1 else 'Normal')


In [19]:
#εκτύπωση των ανωμαλιών
print("Ανωμαλίες στο Test Set:")
print(test_df[test_df['anomaly'] == 1][['flowDuration', 'Label']])

Ανωμαλίες στο Test Set:
        flowDuration    Label
35973     449.587311  Anomaly
151634    128.322163  Anomaly
70144     301.174289  Anomaly
120897    338.127180  Anomaly
26889     182.635001  Anomaly
...              ...      ...
113756    301.127139  Anomaly
113542    422.310526  Anomaly
27937    1138.800492  Anomaly
99953     609.863705  Anomaly
32732     159.497286  Anomaly

[3036 rows x 2 columns]


In [20]:
#κατανομή των προβλέψεων για το σύνολο δοκιμών
total_test_samples = len(test_df)
normal_test_samples = len(test_df[test_df['Label'] == 'Normal'])
anomaly_test_samples = len(test_df[test_df['Label'] == 'Anomaly'])

print(f"\nΣύνολο Δοκιμών (Test Set): {total_test_samples}")
print(f"Φυσιολογικές Προβλέψεις: {normal_test_samples}")
print(f"Ανώμαλες Προβλέψεις: {anomaly_test_samples}")
print(f"Ποσοστό Ανωμαλιών: {(anomaly_test_samples / total_test_samples) * 100:.2f}%")


Σύνολο Δοκιμών (Test Set): 31025
Φυσιολογικές Προβλέψεις: 27989
Ανώμαλες Προβλέψεις: 3036
Ποσοστό Ανωμαλιών: 9.79%
